### Chain example with custom tools

In [ ]:
#setup
import openai
import os
from pprint import pprint
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.chat_models import AzureChatOpenAI
from typing import Optional, Type
from langchain.tools import BaseTool
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

# read local .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

# setup azure openai api
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT")
# openai.api_version = "2023-03-15-preview"
openai.api_version = "2023-05-15"
openai.api_key  = os.getenv('AZURE_OAI_KEY')

llm_name = "gpt-3.5-turbo"
deployment_name_gpt = os.getenv("AZURE_OAI_MODEL_GPT_3")
deployment_name_ada = os.getenv("AZURE_OAI_MODEL_ADA")

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_BASE"] = openai.api_base
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["OPENAI_API_VERSION"] = openai.api_version

In [ ]:
# create llm instance
llm = AzureChatOpenAI(deployment_name=deployment_name_gpt)

In [ ]:
#define embedding and creating the instnace of azure cognitive search
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=deployment_name_ada, chunk_size=1)
index_name: str = "facts-about-snakes"
azure_search_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    semantic_configuration_name="default",
    embedding_function=embeddings.embed_query,
)

In [ ]:
class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to get exisitng data on snakes from azure cognitive seach"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return azure_search_store.similarity_search(
            query=query, 
            k=4,
            search_type="semantic_hybrid"
        )

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
acs_tools = [CustomSearchTool()]
agent = initialize_agent(
    acs_tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
# chain 1
chain_one = agent

In [ ]:
template = """You are a text formatter. Your job is to return below input in the html format\n
Provide a good tite for the input.
Input:
{input}
"""
prompt_template = PromptTemplate(input_variables=["input"], template=template)

In [ ]:
chain_two = LLMChain(llm=llm, prompt=prompt_template) 

In [ ]:
overall_chain = SimpleSequentialChain(
                  chains=[chain_one, chain_two],
                  verbose=True)

In [ ]:
question = "What are the venomous snakes in North Carolina?"

overall_chain.run(question)

In [ ]:
question = "Why do snakes shed their skins?"

overall_chain.run(question)

In [ ]:
question = "List the tips for Preventing Conflicts with Snakes"

overall_chain.run(question)